#### Часть 1: Создадим и обучим модель, которую будем потом использовать при работе чат-бота

In [2]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# загружаем датасет и создаем датафрейм
newsgroups = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)
data = newsgroups.data
target = newsgroups.target

df = pd.DataFrame({'text': data, 'label': target})
df.head() # проверка

,text,label
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4


In [3]:
# переводим текст в числовые векторы

from sklearn.feature_extraction.text import TfidfVectorizer

# инициализируем TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# векторизуем текст из датафрейма
X = vectorizer.fit_transform(df['text'])

# выделяем таргет
y = df['label']

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# разделим датасет на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# инициализируем модель и обучаем ее
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [10]:
from sklearn.metrics import accuracy_score, classification_report

# делаем предсказания не тестовой выборке
y_pred = clf.predict(X_test)

# рассчитываем метрики качества и создаем classification_report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=newsgroups.target_names)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(report)

Accuracy: 0.9163
Classification Report:
                          precision    recall  f1-score   support

             alt.atheism       0.94      0.93      0.93       236
           comp.graphics       0.76      0.88      0.82       287
 comp.os.ms-windows.misc       0.87      0.85      0.86       290
comp.sys.ibm.pc.hardware       0.77      0.81      0.79       285
   comp.sys.mac.hardware       0.91      0.87      0.89       312
          comp.windows.x       0.90      0.87      0.88       308
            misc.forsale       0.83      0.88      0.86       276
               rec.autos       0.92      0.93      0.93       304
         rec.motorcycles       1.00      0.95      0.97       279
      rec.sport.baseball       0.97      0.97      0.97       308
        rec.sport.hockey       0.97      0.97      0.97       309
               sci.crypt       0.99      0.95      0.97       290
         sci.electronics       0.87      0.87      0.87       304
                 sci.med       0.93

In [11]:
# сохраняем модель для последующего использования в чат-боте
import pickle

Pkl_Filename = "clf_SVG_for_tg_bot.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clf, file)

In [12]:
# загружаем сохраненную модель и для чат-бота будем использовать эту модель
with open(Pkl_Filename, 'rb') as file:  
    clf_SVG_for_tg_bot = pickle.load(file)

In [13]:
# напишем функцию, которая будет предсказывать тему сообщения от пользователя
def predict_category(text, model):

    text_vec = vectorizer.transform([text])
    prediction = model.predict(text_vec)
    return newsgroups.target_names[prediction[0]]

In [14]:
# протестируем работу функции предсказания на зaгруженной модели
sample_text = "Scientists announced that they had discovered a new planet."
predicted_category = predict_category(sample_text, clf_SVG_for_tg_bot)
predicted_category

'sci.space'

#### Часть 2: создаем чат-бот и прописываем логику общения с пользователем

##### Тестирование модели производилось на следующих сообщениях пользователя:
1. Yuri Gagarin is the first cosmonaut, he was the first who fly into space
2. Scientists have invented a new way to treat cancer
3. Will Donald Trump be the next president?
4. The Italian national basketball team has reached the semi-finals 
6. Biology is the science that studies the animal world

In [21]:
# создаем чат-бот
import telebot

my_token = 'мой приватный токен' 
bot = telebot.TeleBot(my_token)

In [22]:
# старт диалога с пользователем
@bot.message_handler(commands=['start'])
def handle_text_message(message):
    bot.send_message(message.chat.id, "Hello, send your message, please)")

In [23]:
# получаем сообщение от пользователя и направляем ему ответ: предсказание темы сообщения 
@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    received_text = message.text # получили сообщение от пользователя 
    bot.send_message(message.chat.id, "Thank you. I got a message. Now I'll try to guess the topic.")
    
    predicted_category = predict_category(received_text, clf_SVG_for_tg_bot) # предсказываем категорию

    bot.send_message(message.chat.id, 'Your message is probably on the topic {}'.format(predicted_category))

In [20]:
bot.polling(none_stop=True, interval=0) #обязательная для работы бота часть